In [1]:
import os
import json
import pickle
import numpy as np
import pandas as pd
from utils import *
from three_stage_model import *

pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',500)

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.colors import ListedColormap

%matplotlib inline

### Generates the three default dataframes: df, df_sub, and df_flood

In [2]:
def get_df_for_heuristic():
    df = pd.read_csv("/home1/07346/ashukla/ThreeStageModel/data/192_Scenario/Final_Input1.csv")
    directions = ["w", "wnw", "nw", "nnw", "n", "nne"]
    categories = ["2", "3", "4", "5"]
    forward_speeds = ["05", "10", "15", "25"]
    lister = []
    for i in directions:
        for j in range(len(categories)):
            for k in range(len(forward_speeds)):
                lister.append("max_flood_level_" + i +"_" + categories[j] + "_" + forward_speeds[k])
    df = df[list(df.columns[0:9]) + lister]
    df_sub = df[["SubNum", "load"]].groupby("SubNum").sum()
    df_flood = df[["SubNum"] + lister]
    df_flood = df_flood.drop_duplicates().set_index("SubNum") # drop duplicates
    df_flood = df_flood.loc[(df_flood.sum(axis=1) != 0), :] # drop substations that are not flooded
    """df_sub has load demand for all the substations"""
    """df_flood has only flooded substations"""
    return df, df_sub, df_flood

### Solution Analysis

In [3]:
def model_analysis(main_path):    
    
    with open(main_path + "model_params.json", 'r') as f:
        params = json.load(f)   
    params["path_to_input"] = os.getcwd() + "/data/192_Scenario/"
    
    with open(main_path + "model_scenarios.json", 'r') as f:
        model_scenarios_string = json.load(f)
    
    model_scenarios = {}
    for k in model_scenarios_string:
        model_scenarios[int(k)] = model_scenarios_string[k]
    
    base_model = three_stage_model(params, model_scenarios)
    base_model.model.update()
    sol_path = main_path + "solution.sol"
    base_model.model.read(sol_path)
    base_model.model.update()

    hardening_decisions = {}
    tiger_dam_decisions = {}

    for sub_id in df_flood.index:
        hardening_decisions[sub_id] = round(base_model.x_mit[sub_id].Start*params["mit_level"])
    for sub_id in df_flood.index:
        for j in model_scenarios:
            tiger_dam_decisions[str(sub_id) + "_" + str(j)] = round(base_model.x_prep[sub_id,j].Start*params["prep_level"])
    
    with open(main_path + 'first_stage_decisions.json', 'w') as fp:
        json.dump(hardening_decisions, fp)
    
    with open(main_path + 'second_stage_decisions.json', 'w') as fp:
        json.dump(tiger_dam_decisions, fp)
    
    return hardening_decisions, tiger_dam_decisions, base_model

### Parsing solutions from .sol files and saving them in .json files

In [4]:
df, df_sub, df_flood = get_df_for_heuristic()

In [ ]:
# voll_list = [250,500,1000,2000,3000,4000,5000,6000]

# for i in voll_list:
#     print(i)
#     main_path = "/work2/07346/ashukla/ls6/ThreeStageModel/output/granular_voll_" + str(i) + "/"
#     voll_h, voll_t, voll_b = model_analysis(main_path)

In [5]:
voll_list = [250,500,1000,2000,3000,4000,5000,6000]

for i in voll_list:
    print("VOLL is\t", i)
    
    main_path = "/work2/07346/ashukla/ls6/ThreeStageModel/output/granular_voll_" + str(i) + "/"
    with open(main_path + "first_stage_decisions.json") as f:
        voll_h = json.load(f)
    with open(main_path + "second_stage_decisions.json") as f:
        voll_td = json.load(f)
        
    main_path = "/work2/07346/ashukla/ls6/ThreeStageModel/output/perfect_prediction_" + str(i) + "/"
    with open(main_path + "first_stage_decisions.json") as f:
        hardening = json.load(f)
    with open(main_path + "second_stage_decisions.json") as f:
        td = json.load(f)
    
    
    budget_pp = 0
    budget_v = 0
    counter = 0
    for j in df_flood.index:
        j = str(j)
        budget_pp = budget_pp + hardening[j]
        budget_v = budget_v + voll_h[j]
        
        if voll_h[j] != hardening[j]:
            #print(j, voll_h[j], hardening[j])
            counter = counter + 1
    print("Count is\t", counter)
    print("Budget PP and VOLl are \t", budget_pp, budget_v)
    print("*********************************")
    print("*********************************")
    print("*********************************")
    print("*********************************")

VOLL is	 250
Count is	 7
Budget PP and VOLl are 	 333 330
*********************************
*********************************
*********************************
*********************************
VOLL is	 500
Count is	 2
Budget PP and VOLl are 	 456 450
*********************************
*********************************
*********************************
*********************************
VOLL is	 1000
Count is	 1
Budget PP and VOLl are 	 516 513
*********************************
*********************************
*********************************
*********************************
VOLL is	 2000
Count is	 10
Budget PP and VOLl are 	 594 564
*********************************
*********************************
*********************************
*********************************
VOLL is	 3000
Count is	 5
Budget PP and VOLl are 	 630 597
*********************************
*********************************
*********************************
*********************************
VOLL is	 4000
Count is	 2


In [ ]:
hardening